In [2]:
import pandas as pd
import re
import numpy as np

In [3]:
from datasketch import MinHash, MinHashLSH

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxx"

In [ ]:
!pip install kaggle
!kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024 -p /Users/daryasavitskaya/Desktop/Finding_similar_items/ --unzip

In [4]:
job_sum = pd.read_csv('/Users/daryasavitskaya/Desktop/Finding_similar_items/job_summary.csv')

In [5]:
job_sum.head()

,job_link,job_summary
0,https://www.linkedin.com/jobs/view/restaurant-...,Rock N Roll Sushi is hiring a Restaurant Manag...
1,https://www.linkedin.com/jobs/view/med-surg-re...,Schedule\n: PRN is required minimum 12 hours p...
2,https://www.linkedin.com/jobs/view/registered-...,Description\nIntroduction\nAre you looking for...
3,https://uk.linkedin.com/jobs/view/commercial-a...,Commercial account executive\nSheffield\nFull ...
4,https://www.linkedin.com/jobs/view/store-manag...,Address:\nUSA-CT-Newington-44 Fenn Road\nStore...


In [6]:
job_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297332 entries, 0 to 1297331
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   job_link     1297332 non-null  object
 1   job_summary  1297332 non-null  object
dtypes: object(2)
memory usage: 19.8+ MB


In [7]:
sum_10 = job_sum[:129733]

In [8]:
sum_10.head()

,job_link,job_summary
379379,https://www.linkedin.com/jobs/view/customer-se...,work directly with medical professionals and p...
1164579,https://www.linkedin.com/jobs/view/human-resou...,About RateGain:\nFounded in 2004 and headquart...
599687,https://www.linkedin.com/jobs/view/cna-at-fusi...,Interested in making a positive impact? You’ve...
764362,https://www.linkedin.com/jobs/view/restaurant-...,Are You Craving A Career With An Industry Lead...
523800,https://www.linkedin.com/jobs/view/sioux-falls...,Sioux Falls NCLEX-RN Tutor Jobs\nThe Varsity T...


In [9]:
sum_10['job_summary'] = sum_10['job_summary'].str.lower()

In [10]:
sum_10['job_summary'].iloc[0]

'work directly with medical professionals and physicians to facilitate patient care\nadvocate for and work in lockstep with chronic patients\nwork for an organization that rewards success and fosters a culture of promotion\njob responsibilities include\nprovide outcome-based solutions for our physician referral sources and existing patient population\nsubmit new orders and monitor their completion, from initiation to setup, working closely with the clinical, sales, and service departments\nprepare and analyze the billing and setup paperwork for patient services\nwork closely with existing patients to troubleshoot issues, identify solutions, and dispatch corrective action via the appropriate departments within the organization\nrecommend potential products or services to management by collecting patient information and analyzing customer needs\nactively participate in the training of new customer service and sales personnel\n#csr\nqualifications\nhigh school diploma or ged preferred\npr

In [11]:
sum_10['words'] = sum_10['job_summary'].apply(lambda x: x.split())

In [12]:
sum_10.head()

,job_link,job_summary,words
379379,https://www.linkedin.com/jobs/view/customer-se...,work directly with medical professionals and p...,"[work, directly, with, medical, professionals,..."
1164579,https://www.linkedin.com/jobs/view/human-resou...,about rategain:\nfounded in 2004 and headquart...,"[about, rategain:, founded, in, 2004, and, hea..."
599687,https://www.linkedin.com/jobs/view/cna-at-fusi...,interested in making a positive impact? you’ve...,"[interested, in, making, a, positive, impact?,..."
764362,https://www.linkedin.com/jobs/view/restaurant-...,are you craving a career with an industry lead...,"[are, you, craving, a, career, with, an, indus..."
523800,https://www.linkedin.com/jobs/view/sioux-falls...,sioux falls nclex-rn tutor jobs\nthe varsity t...,"[sioux, falls, nclex-rn, tutor, jobs, the, var..."


In [13]:
#calculating average number of words 
sum_10['words'] = sum_10['words'].apply(lambda x: x if isinstance(x, list) else [])
sum_10['word_count'] = sum_10['words'].apply(len)
average_words = sum_10['word_count'].mean()
average_words

549.5112037800714

In [15]:
sum_10['words'] = sum_10['words'].apply(lambda x: x if isinstance(x, list) else [])

In [16]:
all_words = [word for word_list in sum_10['words'] for word in word_list if not re.search(r'\d', word)]

In [17]:
from collections import Counter

In [18]:
word_counts = Counter(all_words)

In [19]:
word_counts_df = pd.DataFrame(word_counts.items(), columns=['word', 'count'])
word_counts_df = word_counts_df.sort_values(by='count', ascending=False)

In [22]:
word_counts_df.head(30)

,word,count
5,and,3819218
7,to,2147189
50,the,2047029
25,of,1622418
23,a,1227786
13,in,1133241
12,for,854689
2,with,851033
74,or,557435
141,is,538077


In [24]:
stop_words = word_counts_df[:30]
stop_words = set(stop_words['word'])
stop_words

{'a',
 'all',
 'an',
 'and',
 'are',
 'as',
 'at',
 'be',
 'by',
 'experience',
 'for',
 'in',
 'is',
 'more',
 'of',
 'on',
 'or',
 'our',
 'show',
 'team',
 'that',
 'the',
 'this',
 'to',
 'we',
 'will',
 'with',
 'work',
 'you',
 'your'}

In [29]:
rare_words = word_counts_df[word_counts_df['count'] < 200]
rare_words = set(rare_words['word'])
rare_words

{'bhd,',
 '(lhe)',
 'ind-ab',
 '(iras).',
 'excellence);',
 'jws?_',
 'visme',
 'burnaby,',
 'kinovo',
 'bess))',
 'usacareers@primark.com',
 'non-geospatially',
 'location/facility',
 'southfield,',
 'timekeepers’',
 'melbourne/orlando,',
 '(trauma,',
 'shipston-on',
 'contentful,',
 '(advantageous).',
 'na:eun',
 'workplan',
 '(dollar',
 "goal'",
 "sebastian's",
 '●this',
 'vinsetta',
 '&starting',
 'cali',
 'sushi-grade',
 'quotidien.',
 '(keyword,',
 'systemverilog/uvm',
 'environment…',
 'ᓯᕗᓪᓕᐅᑎᒋᐊᖃᖅᐸᑦᑐᓂᑦ/ᐱᓕᕆᐊᖑᒋᐊᖃᖅᐸᑦᑐᓂᑦ',
 'mcc’s',
 'vem.',
 'dishwashers,',
 'full-course',
 'pfep',
 'discharges.',
 'inst',
 'americas_tta@baxter.com',
 'injector).',
 'dit,',
 'unna',
 'extra-help',
 'clinical-reasoning',
 'home/field',
 '(tcavi),',
 'ecmo).',
 'schreibe',
 'seizure.',
 'malpractice/maladministration',
 'tillamook’s',
 'exelon',
 'anyplace.',
 'shimmick’s',
 'action/disability/veterans',
 'ccvt’s,',
 '(aprn)',
 'sceca',
 'pqvc',
 'safety:implement',
 'colonography',
 'dkn)',
 '"ecops

In [30]:
exclude_words_set = stop_words.union(rare_words)

In [31]:
def mh(word_set, exclude_words_set, num_perm=64):
    m = MinHash(num_perm=num_perm)
    filtered_word_set = word_set - exclude_words_set
    for word in filtered_word_set:
        m.update(word.encode('utf8'))
    return m

In [32]:
mhs = []
for words in sum_10['words']:
    word_set = set(words)  
    minhash = mh(word_set, exclude_words_set) 
    mhs.append(minhash)

In [33]:
lsh = MinHashLSH(threshold=0.5, num_perm=64)  

In [34]:
for i, minhash in enumerate(mhs):
    lsh.insert(f"job_{i}", minhash)

In [35]:
similar_pairs = []
for i, minhash in enumerate(minhashes):
    result = lsh.query(minhash)  
    if len(result) > 1:  
        similar_pairs.append((i, result))

In [ ]:
#Results

#for i, j in similar_pairs:
#    print(f"Job {i} and Job {j} are similar.")
#    print(f"Job {i} Summary: {sum_10.iloc[i]['job_summary']}")
#    print(f"Job {j} Summary: {sum_10.iloc[j]['job_summary']}")
#    print('-' * 80)